In [16]:
from urllib.request import urlopen as uReq
from bs4 import BeautifulSoup as soup
from urllib.request import urlopen
import pandas as pdp

In [ ]:
my_url = "<"


In [11]:
uClient = uReq(my_url)
page_html = uClient.read()
uClient.close()

In [13]:
page_html

b'<!DOCTYPE html>\n<html lang="en">\n<head>\n\t<meta charset="UTF-8">\n\t<title>Quotes to Scrape</title>\n    <link rel="stylesheet" href="/static/bootstrap.min.css">\n    <link rel="stylesheet" href="/static/main.css">\n    \n    \n</head>\n<body>\n    <div class="container">\n        <div class="row header-box">\n            <div class="col-md-8">\n                <h1>\n                    <a href="/" style="text-decoration: none">Quotes to Scrape</a>\n                </h1>\n            </div>\n            <div class="col-md-4">\n                <p>\n                \n                    <a href="/login">Login</a>\n                \n                </p>\n            </div>\n        </div>\n    \n\n<div class="row">\n    <div class="col-md-8">\n\n    <div class="quote" itemscope itemtype="http://schema.org/CreativeWork">\n        <span class="text" itemprop="text">\xe2\x80\x9cThe world as we have created it is a process of our thinking. It cannot be changed without changing our thinki

In [ ]:
page_soup = soup(page_html, "html.parser")
page_soup

<!DOCTYPE html>

<html lang="en">
<head>
<meta charset="utf-8"/>
<title>Quotes to Scrape</title>
<link href="/static/bootstrap.min.css" rel="stylesheet"/>
<link href="/static/main.css" rel="stylesheet"/>
</head>
<body>
<div class="container">
<div class="row header-box">
<div class="col-md-8">
<h1>
<a href="/" style="text-decoration: none">Quotes to Scrape</a>
</h1>
</div>
<div class="col-md-4">
<p>
<a href="/login">Login</a>
</p>
</div>
</div>
<div class="row">
<div class="col-md-8">
<div class="quote" itemscope="" itemtype="http://schema.org/CreativeWork">
<span class="text" itemprop="text">“The world as we have created it is a process of our thinking. It cannot be changed without changing our thinking.”</span>
<span>by <small class="author" itemprop="author">Albert Einstein</small>
<a href="/author/Albert-Einstein">(about)</a>
</span>
<div class="tags">
            Tags:
            <meta class="keywords" content="change,deep-thoughts,thinking,world" itemprop="keywords"/>
<a class="

In [19]:
# trouve la première div avec cette class
div = page_soup.find("div", class_="col-md-8")
print(div.text.strip())

Quotes to Scrape


In [53]:
import pandas as pd
from urllib.request import urlopen
from bs4 import BeautifulSoup

data = []

page = 1
while True:
    url = f"https://quotes.toscrape.com/page/{page}/"
    html = urlopen(url).read()
    soup = BeautifulSoup(html, "html.parser")

    # Récupérer toutes les citations sur la page
    quotes = soup.find_all("div", class_="quote")

    # Si aucune citation => fin du site => on stoppe la boucle
    if len(quotes) == 0:
        break

    for q in quotes:
        text = q.find("span", class_="text").text.strip()
        author = q.find("small", class_="author").text.strip()
        tags = [tag.text for tag in q.find_all('a', class_='tag')]
        data.append([text, author, tags])

    page += 1  # page suivante



In [61]:
df = pd.DataFrame(data, columns=["Quote", "Author", "Tags"])
df['id'] = range(1, len(df) + 1)
# Reorder columns to put 'ID' first
df = df[['id'] + [col for col in df.columns if col != 'id']]
df


,id,Quote,Author,Tags
0,1,“The world as we have created it is a process ...,Albert Einstein,"[change, deep-thoughts, thinking, world]"
1,2,"“It is our choices, Harry, that show what we t...",J.K. Rowling,"[abilities, choices]"
2,3,“There are only two ways to live your life. On...,Albert Einstein,"[inspirational, life, live, miracle, miracles]"
3,4,"“The person, be it gentleman or lady, who has ...",Jane Austen,"[aliteracy, books, classic, humor]"
4,5,"“Imperfection is beauty, madness is genius and...",Marilyn Monroe,"[be-yourself, inspirational]"
...,...,...,...,...
95,96,“You never really understand a person until yo...,Harper Lee,[better-life-empathy]
96,97,“You have to write the book that wants to be w...,Madeleine L'Engle,"[books, children, difficult, grown-ups, write,..."
97,98,“Never tell the truth to people who are not wo...,Mark Twain,[truth]
98,99,"“A person's a person, no matter how small.”",Dr. Seuss,[inspirational]


In [62]:
df.to_json("quotes.json", orient="records", force_ascii=False)